In [ ]:
import pandas as pd
import matplotlib.pyplot as plt 
from scipy import stats

In [ ]:
genes = pd.read_csv("hg19.refGene.gtf.gz",sep="\t",header=None)

In [ ]:
region=("chr1",1000000,2000000)
genes_1mb = genes.loc[(genes[0]=="chr1")&(genes[3]<region[2])&(genes[4]>region[1])]
genes_1mb

In [ ]:
genes_1mb.loc[:,"name"] = genes_1mb[8].str.split(";").str[-2].str.replace("gene_name ","").str.strip(' ').str.strip('"')
genes_1mb.loc[:,"transcript"] = genes_1mb[8].str.split(";").str[1]
genes_1mb

In [ ]:
exons = genes_1mb.loc[genes_1mb[2]=="exon",:]

In [ ]:
uniq_gene_region = genes_1mb.groupby("name").agg({3:"min",4:"max",6:stats.mode})
uniq_gene_region

In [ ]:
uniq_gene_region = uniq_gene_region.reset_index()
flank = 0.05 * (region[2] - region[1])
uniq_gene_region["left"]=uniq_gene_region[3]-flank
uniq_gene_region["right"]=uniq_gene_region[4]+flank
uniq_gene_region

In [ ]:
stacks=[]
stack_dic={}
for index,row in uniq_gene_region.reset_index().sort_values([3]).iterrows():
    if len(stacks)==0:
        stacks.append([(row["left"],row["right"])])
        stack_dic[row["name"]] = 0
    else:
        for i in range(len(stacks)):
            for j in range(len(stacks[i])):
                # if overlap
                if (row["left"]>stacks[i][j][0] and row["left"]<stacks[i][j][1]) or (row["right"]>stacks[i][j][0] and row["right"]<stacks[i][j][1]):
                    # if not last stack : break
                    if i<len(stacks)-1:
                        break
                    # if last stack : add a new stack
                    else:
                        stacks.append([(row["left"],row["right"])])
                        stack_dic[row["name"]] = i+1
                        break
                # if no overlap       
                else:
                    # not last in a stack
                    if j<len(stacks[i])-1:
                        #if in the middle
                        if row["left"]>stacks[i][j][1] and row["right"]<stacks[i][j+1][0]:
                            stacks[i].insert(j+1,(row["left"],row["right"]))
                            stack_dic[row["name"]] = i
                            break
                    #last one in a stack
                    elif row["left"]>stacks[i][j][1]:
                        stacks[i].append((row["left"],row["right"]))
                        stack_dic[row["name"]] = i
                        break
            if row["name"] in stack_dic.keys():
                break         
uniq_gene_region["stack"] = -uniq_gene_region["name"].map(stack_dic)


In [ ]:
exons["stack"] = -exons["name"].map(stack_dic)
exons

In [ ]:
fig = plt.figure(figsize=(20,5))
ax = fig.add_subplot(1,1,1)
for index,row in uniq_gene_region.reset_index().iterrows():
    if row[6][0]=="+":
        gene_anno = row["name"] + "->"
    else:
        gene_anno = "<-" + row["name"] 
    ax.plot((row[3],row[4]),(row["stack"]*4,row["stack"]*4),color="blue")
    ax.text(x=(row[3]+row[4])/2,y=row["stack"]*4-1.5,s=gene_anno,ha="center",va="top",color="black")
for index,row in exons.iterrows():
    ax.plot((row[3],row[4]),(row["stack"]*4,row["stack"]*4),linewidth=10,color="blue")
#ax.set_ylim((-4,uniq_gene_region["stack"].max()*4+4))